In [1]:
pip install pymodbus

Note: you may need to restart the kernel to use updated packages.


In [16]:
sudo chmod 666 /dev/ttyUSB0

SyntaxError: invalid syntax (1467591312.py, line 1)

In [17]:
import serial
from pymodbus.pdu import ModbusRequest
from pymodbus.client.sync import ModbusSerialClient as ModbusClient
from pymodbus.transaction import ModbusRtuFramer
from pymodbus.constants import Defaults

from time import sleep
from typing import List
import struct


# Константы повторного опроса при пропаже соединенния или пакета
Defaults.RetryOnEmpty = True
Defaults.Timeout = 5
Defaults.Retries = 5

In [18]:
client = ModbusClient(method = 'rtu', 
                    port='/dev/ttyUSB0', 
                    baudrate=115200,
                    timeout=1, 
                    bytesize=8, 
                    stopbits=1)

vacuum_system_slave_id = 1

In [27]:
# кодирование и декодирование float -> [uint16, uint16]
def encode_float(value: float) -> List:
	float_in_bytes = bytearray(struct.pack("f", value))
	values = [] 
	for idx in range(0, len(float_in_bytes), 2): 
		values.append((float_in_bytes[idx]<<8) + float_in_bytes[idx + 1]) 
	
	return values 

def decode_to_float(data: List) -> float:
	barray = bytearray([]) 
	for value in data: 
		barray += bytearray([value>>8, value&0xFF]) 
	
	if isinstance(barray, list): 
		float_raw = struct.unpack("f", bytearray(barray))[0] 
	else:
		float_raw =  struct.unpack("f", barray)[0]

	return round(float_raw, 4)


def mb_connect():
    # првоеряем подключение, пытаемся подключиться 10 раз
    for i in range(10):
        connection = client.connect()
        if(not connection):
            print("Modbus not connected... try to connect again")
            sleep(0.1)
        else:
            print("Modbus connected succesfully")
            return connection

    return False

class VacuumModbusManager(object):
    def mb_read_state(self):
        connection = mb_connect()
        if not connection:
            return False, None

        state = client.read_holding_registers(0, 30, unit=vacuum_system_slave_id) # start_address, count, slave_id
        
        if state.isError():
            print('Unable to read state or there is the connection problem.')
            return False, None

        return True, state.registers


    def mb_read_pressure_state(self):
        connection = mb_connect()
        if not connection:
            return False, None

        state = client.read_holding_registers(22, 8, unit=vacuum_system_slave_id) # start_address, count, slave_id
        
        if state.isError():
            print('Unable to read state or there is the connection problem.')
            return False, None

        # return True, state.registers

        # convert raw bytes to float
        data = []
        data.append(decode_to_float(state.registers[0:2]))
        data.append(decode_to_float(state.registers[2:4]))
        data.append(decode_to_float(state.registers[4:6]))
        data.append(decode_to_float(state.registers[6:8]))

        return True, data


    def mb_write_default_params(self):
        connection = mb_connect()
        if not connection:
            return False
        
        default_parameters = [600, 200, 850, 1000, 20000, 50]

        # записываем регистры и проверяем ответ
        # TODO: сделать несколько попыток
        response = client.write_registers(4, default_parameters, unit=vacuum_system_slave_id)
        if response.isError():
            print('Unable to write default params or there is the connection problem.')
            return False
        return True


    def mb_write_single_register(self, adress, data):
        connection = mb_connect()
        if not connection:
            return False
    
        response = client.write_register(adress, int(data), unit=vacuum_system_slave_id)  
        if response.isError():
            print('Unable to write command or there is the connection problem.')
            return False

        return True

# 
vacuum_mb = VacuumModbusManager()

## Conection

In [20]:
mb_connect()

Modbus connected succesfully


True

## Read State

In [ ]:
is_connect, state = vacuum_mb.mb_read_state()
state

## Показания датчиков

In [28]:
is_connect, state = vacuum_mb.mb_read_pressure_state()
state

Modbus connected succesfully


[1401, 30020, 59842, 26436, 40012, 34628, 17730, 32068]